In [151]:
import pandas as pd
import os
import boto3
import json
from dotenv import load_dotenv  
from config_part3 import BLS_BUCKET_NAME, API_BUCKET_NAME, API_PREFIX, BLS_PREFIX

In [152]:
# Load environment variables from .env file
load_dotenv()

True

In [153]:
AWS_ACCESS_KEY_ID = os.getenv("BLS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("BLS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION = os.getenv("BLS_DEFAULT_REGION")

In [148]:
# Initialize S3 client with credentials
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
    #region_name=BLS_DEFAULT_REGION
)

In [149]:
# Load part 1 data into df
part1_data = s3.get_object(Bucket=BLS_BUCKET_NAME, Key=f"{BLS_PREFIX}")
part1_df = pd.read_csv(part1_data['Body'], sep='\t')

In [150]:
part1_df.to_csv("output_part1.csv", index=False)

In [154]:
display(part1_df.head())

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [155]:
# Load part 2 data into df
api_population = s3.get_object(Bucket=API_BUCKET_NAME, Key=API_PREFIX)
population_json = json.load(api_population["Body"])
part2_df = pd.json_normalize(population_json['data'])

In [157]:
display(part2_df.head(5))

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0


In [135]:
part2_df.dtypes

Nation ID      object
Nation         object
Year            int64
Population    float64
dtype: object

In [136]:
part2_df['Year'] = part2_df['Year'].astype(int)

In [137]:
df_pop_2013_2018 = part2_df[(part2_df['Year'] >= 2013) & (part2_df['Year'] <= 2018)]

In [138]:
mean_pop = round(df_pop_2013_2018['Population'].mean(),2)
std_pop = round(df_pop_2013_2018['Population'].std(),2)

# round to 2 decimal places
print("Mean Population (2013–2018):", mean_pop)
print("Standard Deviation:", std_pop)

Mean Population (2013–2018): 322069808.0
Standard Deviation: 4158441.04


In [139]:
part1_df.columns = part1_df.columns.str.strip()

In [140]:
part1_df.dtypes


series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object

In [141]:
grouped = part1_df.groupby(['series_id', 'year'])['value'].sum().reset_index()
best_years = grouped.loc[grouped.groupby('series_id')['value'].idxmax()].reset_index(drop=True)


display(best_years)

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2024,860.838
279,PRS88003201,2022,37.200
280,PRS88003202,2022,28.700


In [142]:
# Filter for series_id and period
part1_df['series_id'] = part1_df['series_id'].str.strip()
part1_df['period'] = part1_df['period'].str.strip()
filtered = part1_df[(part1_df['series_id'] == 'PRS30006032') & (part1_df['period'] == 'Q01')]

# Prepare population data
pop_lookup = part2_df[['Year', 'Population']].drop_duplicates()
pop_lookup.columns = ['year', 'Population']

# Merge
final = pd.merge(filtered, pop_lookup, on='year', how='right')
final_result = final[['series_id', 'year', 'period', 'value', 'Population']]
display(final_result)

,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,0.5,316128839.0
1,PRS30006032,2014,Q01,-0.1,318857056.0
2,PRS30006032,2015,Q01,-1.7,321418821.0
3,PRS30006032,2016,Q01,-1.4,323127515.0
4,PRS30006032,2017,Q01,0.9,325719178.0
5,PRS30006032,2018,Q01,0.5,327167439.0
6,PRS30006032,2019,Q01,-1.6,328239523.0
7,PRS30006032,2021,Q01,0.7,331893745.0
8,PRS30006032,2022,Q01,5.3,333287562.0
9,PRS30006032,2023,Q01,0.3,334914896.0
